In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [4]:
%load_ext sql
%sql sqlite:///store.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Оконные функции

Подробное описание оконных функций в SQLite можно посмотреть [на офф. сайте](https://sqlite.org/windowfunctions.html)

**Оконная функция** - это SQL функция, где входящие
значения берутся из "окна" (партиции) одной или более строк в
результирующее выражение SELECT.
Окно специфицируется в блоке OVER( ).
Оконные функции подразделяются на

агрегирующие *Aggregate*,

ранжирующие *Ranking*

и функции смещения *Value*


Определение окна может включать в себя выражение PARTITION BY. В таком случае все строки делятся на группы (партиции) с одними и теми же значениями в выражении PARTITION BY, и тогда оконная функция выполняется раздельно для каждой партиции.

## Aggregate
Знакомые нам SUM, AVG, COUNT, MIN, MAX

In [30]:
%%sql
SELECT Name, UnitPrice, AlbumId,
    sum(UnitPrice) over (partition by AlbumId) as sum_price,
    avg(UnitPrice) over (partition by AlbumId) as avg_price,
    min(UnitPrice) over (partition by AlbumId) as min_price,
    max(UnitPrice) over (partition by AlbumId) as max_price
FROM tracks LIMIT 20;

 * sqlite:///store.db
Done.


Name,UnitPrice,AlbumId,sum_price,avg_price,min_price,max_price
For Those About To Rock (We Salute You),1.99,1,22.9,2.29,0.99,3.49
Put The Finger On You,3.49,1,22.9,2.29,0.99,3.49
Let's Get It Up,2.99,1,22.9,2.29,0.99,3.49
Inject The Venom,1.99,1,22.9,2.29,0.99,3.49
Snowballed,3.49,1,22.9,2.29,0.99,3.49
Evil Walks,0.99,1,22.9,2.29,0.99,3.49
C.O.D.,2.99,1,22.9,2.29,0.99,3.49
Breaking The Rules,0.99,1,22.9,2.29,0.99,3.49
Night Of The Long Knives,0.99,1,22.9,2.29,0.99,3.49
Spellbound,2.99,1,22.9,2.29,0.99,3.49


Для каждой даты, в которую был нанят сотрудник, вывести количество нанятых сотрудников до этой даты включительно.

In [29]:
%%sql

SELECT distinct hiredate,
    count(*) over (order by date(hiredate)) cntHired
from employees;

 * sqlite:///store.db
Done.


HireDate,cntHired
2021-04-01 00:00:00,1
2021-05-01 00:00:00,2
2021-08-14 00:00:00,3
2022-05-03 00:00:00,4
2022-10-17 00:00:00,6
2023-01-02 00:00:00,7
2023-03-04 00:00:00,8


Для каждого трека вывести его отклонение от средней цены трека его альбома

In [38]:
%%sql
SELECT Name, UnitPrice, AlbumId,
    ROUND(AVG(UnitPrice) over (partition by AlbumId),2) as avg_price,
    ROUND(ABS(UnitPrice - AVG(UnitPrice) over (partition by AlbumId)),2) as deviation
FROM tracks LIMIT 20;

 * sqlite:///store.db
Done.


Name,UnitPrice,AlbumId,avg_price,deviation
For Those About To Rock (We Salute You),1.99,1,2.29,0.3
Put The Finger On You,3.49,1,2.29,1.2
Let's Get It Up,2.99,1,2.29,0.7
Inject The Venom,1.99,1,2.29,0.3
Snowballed,3.49,1,2.29,1.2
Evil Walks,0.99,1,2.29,1.3
C.O.D.,2.99,1,2.29,0.7
Breaking The Rules,0.99,1,2.29,1.3
Night Of The Long Knives,0.99,1,2.29,1.3
Spellbound,2.99,1,2.29,0.7


##row_number(), rank(), dense_rank()
Функция row_number() присваивает монотонно
увеличивающееся целое число для каждого строки в
возникающем окне.

In [24]:
%%sql
SELECT a.albumid, a.title,
      row_number()
          OVER (ORDER BY t.name) rn1,
      row_number()
          OVER (ORDER BY t.Milliseconds ASC) rn2,
      t.unitprice AS price,
      t.name AS track_name,
      t.Milliseconds
    FROM tracks t
    JOIN albums a
    ON a.albumid = t.albumid
    WHERE a.albumid = 100;

 * sqlite:///store.db
Done.


AlbumId,Title,rn1,rn2,price,track_name,Milliseconds
100,Iron Maiden,1,4,2.99,01 - Prowler,236173
100,Iron Maiden,2,1,1.99,02 - Sanctuary,196284
100,Iron Maiden,3,7,1.99,03 - Remember Tomorrow,328620
100,Iron Maiden,4,2,1.99,04 - Running Free,197276
100,Iron Maiden,5,9,1.99,05 - Phantom of the Opera,428016
100,Iron Maiden,6,6,1.99,06 - Transylvania,259343
100,Iron Maiden,7,8,1.99,07 - Strange World,332460
100,Iron Maiden,8,5,1.99,08 - Charlotte the Harlot,252708
100,Iron Maiden,9,3,3.99,09 - Iron Maiden,216058


In [32]:
%%sql
SELECT a.albumid,
      rank()
          over (order by a.albumid asc) rank,
      dense_rank()
          over (order by a.albumid asc) dense,
      row_number()
          over (order by a.albumid asc) rownumber,
      a.title,
      t.name as tname
    from tracks t
    join albums a
    ON a.albumid = t.albumid
LIMIT 25;

 * sqlite:///store.db
Done.


AlbumId,rank,dense,rownumber,Title,tname
1,1,1,1,For Those About To Rock We Salute You,For Those About To Rock (We Salute You)
1,1,1,2,For Those About To Rock We Salute You,Put The Finger On You
1,1,1,3,For Those About To Rock We Salute You,Let's Get It Up
1,1,1,4,For Those About To Rock We Salute You,Inject The Venom
1,1,1,5,For Those About To Rock We Salute You,Snowballed
1,1,1,6,For Those About To Rock We Salute You,Evil Walks
1,1,1,7,For Those About To Rock We Salute You,C.O.D.
1,1,1,8,For Those About To Rock We Salute You,Breaking The Rules
1,1,1,9,For Those About To Rock We Salute You,Night Of The Long Knives
1,1,1,10,For Those About To Rock We Salute You,Spellbound


In [33]:
%%sql
SELECT a.albumid, a.title,
      row_number()
          over (partition by a.albumid order by t.name asc) rn,
      t.name as tname
    FROM tracks t
    JOIN albums a
    ON a.albumid = t.albumid
LIMIT 25;

 * sqlite:///store.db
Done.


AlbumId,Title,rn,tname
1,For Those About To Rock We Salute You,1,Breaking The Rules
1,For Those About To Rock We Salute You,2,C.O.D.
1,For Those About To Rock We Salute You,3,Evil Walks
1,For Those About To Rock We Salute You,4,For Those About To Rock (We Salute You)
1,For Those About To Rock We Salute You,5,Inject The Venom
1,For Those About To Rock We Salute You,6,Let's Get It Up
1,For Those About To Rock We Salute You,7,Night Of The Long Knives
1,For Those About To Rock We Salute You,8,Put The Finger On You
1,For Those About To Rock We Salute You,9,Snowballed
1,For Those About To Rock We Salute You,10,Spellbound


Для каждого альбома вывести самый короткий трек

In [11]:
%%sql

select * from
(
select a.albumid, a.title,
      row_number()
          over (partition by a.albumid order by t.Milliseconds asc) rn,
      t.name as tname,
      t.Milliseconds
    from tracks t
    join albums a
    on a.albumid = t.albumid
)
where rn = 1
LIMIT 10;

 * sqlite:///store.db
Done.


albumid,title,rn,tname,Milliseconds
1,For Those About To Rock We Salute You,1,C.O.D.,199836
2,Balls to the Wall,1,Balls to the Wall,342562
3,Restless and Wild,1,Fast As a Shark,230619
4,Let There Be Rock,1,Dog Eat Dog,215196
5,Big Ones,1,Deuces Are Wild,215875
6,Jagged Little Pill,1,Right Through You,176117
7,Facelift,1,We Die Young,152084
8,Warner 25 Anos,1,Outra Vez,126511
9,Plays Metallica By Four Cellos,1,Enter Sandman,221701
10,Audioslave,1,Exploder,206053


То же самое через first_value()

In [12]:
%%sql
select distinct
      a.albumid, a.title,
      first_value(t.name)
          over (partition by a.albumid order by t.Milliseconds asc) tname,
      first_value(t.Milliseconds)
          over (partition by a.albumid order by t.Milliseconds asc) length
    from tracks t
    join albums a
    on a.albumid = t.albumid
LIMIT 10;

 * sqlite:///store.db
Done.


AlbumId,Title,tname,length
1,For Those About To Rock We Salute You,C.O.D.,199836
2,Balls to the Wall,Balls to the Wall,342562
3,Restless and Wild,Fast As a Shark,230619
4,Let There Be Rock,Dog Eat Dog,215196
5,Big Ones,Deuces Are Wild,215875
6,Jagged Little Pill,Right Through You,176117
7,Facelift,We Die Young,152084
8,Warner 25 Anos,Outra Vez,126511
9,Plays Metallica By Four Cellos,Enter Sandman,221701
10,Audioslave,Exploder,206053


In [13]:
%%sql

select * from
(
select a.albumid, a.title,
      row_number()
          over (partition by a.albumid order by t.Milliseconds asc) rn,
      t.name as tname,
      t.Milliseconds
    from tracks t
    join albums a
    on a.albumid = t.albumid
)
where rn = 1
LIMIT 10;

 * sqlite:///store.db
Done.


albumid,title,rn,tname,Milliseconds
1,For Those About To Rock We Salute You,1,C.O.D.,199836
2,Balls to the Wall,1,Balls to the Wall,342562
3,Restless and Wild,1,Fast As a Shark,230619
4,Let There Be Rock,1,Dog Eat Dog,215196
5,Big Ones,1,Deuces Are Wild,215875
6,Jagged Little Pill,1,Right Through You,176117
7,Facelift,1,We Die Young,152084
8,Warner 25 Anos,1,Outra Vez,126511
9,Plays Metallica By Four Cellos,1,Enter Sandman,221701
10,Audioslave,1,Exploder,206053


Таблица зарплат

In [15]:
%%sql
DROP TABLE IF EXISTS EmployeeSalary;
CREATE TABLE EmployeeSalary(EmployeeId integer, salary_date date, salary money);

insert into EmployeeSalary values (1, '2003-01-01', 80000);
insert into EmployeeSalary values (1, '2003-02-01', 100000);
insert into EmployeeSalary values (1, '2003-03-01', 60000);
insert into EmployeeSalary values (1, '2003-04-01', 50000);
insert into EmployeeSalary values (1, '2003-05-01', 100000);
insert into EmployeeSalary values (1, '2003-06-01', 90000);
insert into EmployeeSalary values (2, '2003-01-01', 100000);
insert into EmployeeSalary values (2, '2003-02-01', 110000);
insert into EmployeeSalary values (2, '2003-03-01', 60000);
insert into EmployeeSalary values (2, '2003-04-01', 80000);
insert into EmployeeSalary values (2, '2003-05-01', 90000);
insert into EmployeeSalary values (2, '2003-06-01', 40000);

 * sqlite:///store.db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [16]:
%%sql
Select * from EmployeeSalary;

 * sqlite:///store.db
Done.


EmployeeId,salary_date,salary
1,2003-01-01,80000
1,2003-02-01,100000
1,2003-03-01,60000
1,2003-04-01,50000
1,2003-05-01,100000
1,2003-06-01,90000
2,2003-01-01,100000
2,2003-02-01,110000
2,2003-03-01,60000
2,2003-04-01,80000


Нарастающий итог

In [17]:
%%sql
select s.*,
      sum(salary) over (partition by EmployeeId
                        order by salary_date) sum_salary,
      avg(salary) over (partition by EmployeeId
                        order by salary_date
                        rows between unbounded preceding and current row) average_salary
  from EmployeeSalary s;

 * sqlite:///store.db
Done.


EmployeeId,salary_date,salary,sum_salary,average_salary
1,2003-01-01,80000,80000,80000.0
1,2003-02-01,100000,180000,90000.0
1,2003-03-01,60000,240000,80000.0
1,2003-04-01,50000,290000,72500.0
1,2003-05-01,100000,390000,78000.0
1,2003-06-01,90000,480000,80000.0
2,2003-01-01,100000,100000,100000.0
2,2003-02-01,110000,210000,105000.0
2,2003-03-01,60000,270000,90000.0
2,2003-04-01,80000,350000,87500.0


## Смещение LAG() и LEAD()
LAG() - предыдущее значение

LEAD() - следующее значение

In [41]:
%%sql

SELECT * ,
lag(InvoiceDate) over (order by InvoiceDate) as previous_invoice,
lead(InvoiceDate) over (order by InvoiceDate) as next_invoice
from invoices limit 5;

 * sqlite:///store.db
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,previous_invoice,next_invoice
1,2,2023-01-02 03:56:47,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,None,None,2023-01-03 03:43:33
2,4,2023-01-03 03:43:33,Ullevålsveien 14,Oslo,None,Norway,0171,None,2023-01-02 03:56:47,2023-01-04 08:03:04
3,8,2023-01-04 08:03:04,Grétrystraat 63,Brussels,None,Belgium,1000,None,2023-01-03 03:43:33,2023-01-07 23:29:33
4,14,2023-01-07 23:29:33,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,None,2023-01-04 08:03:04,2023-01-08 10:08:43
5,23,2023-01-08 10:08:43,69 Salem Street,Boston,MA,USA,2113,None,2023-01-07 23:29:33,2023-01-08 13:25:49


Для каждого клиента рядом с его заказом вывести дату предыдущего

In [46]:
%%sql

SELECT InvoiceId, CustomerId, InvoiceDate,
lag(InvoiceDate) over (partition by CustomerId order by InvoiceDate) as previous_invoice
from invoices limit 15;

 * sqlite:///store.db
Done.


InvoiceId,CustomerId,InvoiceDate,previous_invoice
98,1,2023-05-31 03:20:30,None
121,1,2023-07-10 16:37:01,2023-05-31 03:20:30
143,1,2023-08-08 20:15:25,2023-07-10 16:37:01
195,1,2023-10-14 16:40:38,2023-08-08 20:15:25
316,1,2024-04-24 09:01:45,2023-10-14 16:40:38
327,1,2024-05-04 14:00:44,2024-04-24 09:01:45
382,1,2024-07-27 23:10:04,2024-05-04 14:00:44
1,2,2023-01-02 03:56:47,None
12,2,2023-01-29 11:51:57,2023-01-02 03:56:47
67,2,2023-04-09 10:44:25,2023-01-29 11:51:57


## Спецификация окна (frame-spec)
frame-spec определяет, какие строки считываются агрегатной
оконной функцией

Спецификация окна по умолчанию:

> `RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`

Значение по умолчанию означает, что агрегатная оконная
функция считывает все строки по умолчанию от начала
раздела до текущей строки, совпадающие с ней в рамках
сортировки

In [27]:
%%sql
Select s.*,
    sum(salary) over (partition by EmployeeId
                        order by salary_date asc
                        rows between unbounded preceding and current row) as sum_up,
    sum(salary) over (partition by EmployeeId
                        order by salary_date asc
                        rows between unbounded preceding and unbounded following) as sum_full,
    sum(salary) over (partition by EmployeeId
                        order by salary_date asc
                        rows between 2 preceding and current row ) as sum_minus_2,
    first_value(salary) over (partition by EmployeeId
                        order by salary_date asc
                        rows between 2 preceding and current row ) as fv_minus_2
from EmployeeSalary s;

 * sqlite:///store.db
Done.


EmployeeId,salary_date,salary,sum_up,sum_full,sum_minus_2,fv_minus_2
1,2003-01-01,80000,80000,480000,80000,80000
1,2003-02-01,100000,180000,480000,180000,80000
1,2003-03-01,60000,240000,480000,240000,80000
1,2003-04-01,50000,290000,480000,210000,100000
1,2003-05-01,100000,390000,480000,210000,60000
1,2003-06-01,90000,480000,480000,240000,50000
2,2003-01-01,100000,100000,480000,100000,100000
2,2003-02-01,110000,210000,480000,210000,100000
2,2003-03-01,60000,270000,480000,270000,100000
2,2003-04-01,80000,350000,480000,250000,110000
